In [ ]:
#default_exp base_constructor

Cells will be exported to model_constructor.constructor,
unless a different module is specified after an export flag: `%nbdev_export special.module`


# model_constructor

> Create pytorch model.

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.test import *
import torch


In [ ]:
#export
import torch.nn as nn
from collections import OrderedDict
from model_constructor.layers import ConvLayer, Noop, Flatten

In [ ]:
#export
act_fn = nn.ReLU(inplace=True)

# Stem

In [ ]:
#export
class Stem(nn.Sequential):
    """Base stem"""

    def __init__(self, c_in=3, stem_sizes=[], stem_out=64,
                 conv_layer=ConvLayer, stride_on=0,
                 stem_bn_last=False, bn_1st=True,
                 stem_use_pool=True, stem_pool=nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
                 **kwargs
                 ):
        self.sizes = [c_in] + stem_sizes + [stem_out]
        num_layers = len(self.sizes) - 1
        stem = [(f"conv_{i}", conv_layer(self.sizes[i], self.sizes[i + 1],
                stride=2 if i == stride_on else 1, act=True,
                bn_layer=not stem_bn_last if i == num_layers - 1 else True,
                bn_1st=bn_1st, **kwargs))
                for i in range(num_layers)]
        if stem_use_pool:
            stem += [('pool', stem_pool)]
        if stem_bn_last:
            stem.append(('bn', nn.BatchNorm2d(stem_out)))
        super().__init__(OrderedDict(stem))

    def extra_repr(self):
        return f"sizes: {self.sizes}"

In [ ]:
stem = Stem()
stem

Stem(
  sizes: [3, 64]
  (conv_0): ConvLayer(
    (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
)

In [ ]:
xb = torch.randn(8, 3, 128, 128)
y = stem(xb)
y.shape

torch.Size([8, 64, 32, 32])

In [ ]:
stem = Stem(use_bn=True)
stem

Stem(
  sizes: [3, 64]
  (conv_0): ConvLayer(
    (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
)

In [ ]:
xb = torch.randn(8, 3, 128, 128)
y = stem(xb)
y.shape

torch.Size([8, 64, 32, 32])

In [ ]:
stem = Stem(use_bn=True, stem_sizes=[32], stride_on=1)
stem

Stem(
  sizes: [3, 32, 64]
  (conv_0): ConvLayer(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (conv_1): ConvLayer(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
)

In [ ]:
xb = torch.randn(8, 3, 128, 128)
y = stem(xb)
y.shape

torch.Size([8, 64, 32, 32])

In [ ]:
assert y.shape, torch.Size([64, 64, 32, 32])

In [ ]:
stem = Stem(use_bn=True, stem_sizes=[32], stride_on=1, bn_1st=False)
stem

Stem(
  sizes: [3, 32, 64]
  (conv_0): ConvLayer(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (act_fn): ReLU(inplace=True)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_1): ConvLayer(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (act_fn): ReLU(inplace=True)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
)

# Body

## Blocks

In [ ]:
#export
def DownsampleBlock(conv_layer, ni, nf, ks, stride, act=False, **kwargs):
    '''Base downsample for res-like blocks'''
    return conv_layer(ni, nf, ks, stride, act, **kwargs)

In [ ]:
#export
class BasicBlock(nn.Module):
    """Basic block (simplified) as in pytorch resnet"""
    def __init__(self, ni, nf, expansion=1, stride=1, zero_bn=False,
                 conv_layer=ConvLayer, act_fn=act_fn,
                 downsample_block=DownsampleBlock, **kwargs):
        super().__init__()
        self.downsample = not ni == nf or stride == 2
        self.conv = nn.Sequential(OrderedDict([
            ('conv_0', conv_layer(ni, nf, stride=stride, act_fn=act_fn, **kwargs)),
            ('conv_1', conv_layer(nf, nf, zero_bn=zero_bn, act=False, act_fn=act_fn, **kwargs))]))
        if self.downsample:
            self.downsample = downsample_block(conv_layer, ni, nf, ks=1, stride=stride, act=False, **kwargs)
        self.merge = Noop()
        self.act_conn = act_fn

    def forward(self, x):
        identity = x
        out = self.conv(x)
        if self.downsample:
            identity = self.downsample(x)
        return self.act_conn(self.merge(out + identity))

In [ ]:
#export
class Bottleneck(nn.Module):
    '''Bottlneck block for resnet models'''
    def __init__(self, ni, nh, expansion=4, stride=1, zero_bn=False,
                 conv_layer=ConvLayer, act_fn=act_fn,
                 downsample_block=DownsampleBlock, **kwargs):
        super().__init__()
        self.downsample = not ni == nh or stride == 2
        ni = ni * expansion
        nf = nh * expansion
        self.conv = nn.Sequential(OrderedDict([
            ('conv_0', conv_layer(ni, nh, ks=1,            act_fn=act_fn, **kwargs)),   # noqa: E241
            ('conv_1', conv_layer(nh, nh, stride=stride,   act_fn=act_fn, **kwargs)),   # noqa: E241
            ('conv_2', conv_layer(nh, nf, ks=1, zero_bn=zero_bn, act=False, act_fn=act_fn, **kwargs))]))
        if self.downsample:
            self.downsample = downsample_block(conv_layer, ni, nf, ks=1,
                                               stride=stride, act=False, act_fn=act_fn, **kwargs)
        self.merge = Noop()
        self.act_conn = act_fn

    def forward(self, x):
        identity = x
        out = self.conv(x)
        if self.downsample:
            identity = self.downsample(x)
        return self.act_conn(self.merge(out + identity))

In [ ]:
#export
class BasicLayer(nn.Sequential):
    '''Layer from blocks'''
    def __init__(self, block, blocks, ni, nf, expansion, stride, sa=False, **kwargs):
        self.ni = ni
        self.nf = nf
        self.blocks = blocks
        self.expansion = expansion
        super().__init__(OrderedDict(
            [(f'block_{i}', block(ni if i == 0 else nf, nf, expansion,
                                  stride if i == 0 else 1,
                                  sa=sa if i == blocks - 1 else False,
                                  **kwargs))
             for i in range(blocks)]))

    def extra_repr(self):
        return f'from {self.ni * self.expansion} to {self.nf}, {self.blocks} blocks, expansion {self.expansion}.'

## Body constructor

In [ ]:
#export
class Body(nn.Sequential):
    '''Constructor for body'''
    def __init__(self, block,
                 body_in=64, body_out=512,
                 bodylayer=BasicLayer, expansion=1,
                 layer_szs=[64, 128, 256, ], blocks=[2, 2, 2, 2],
                 sa=False, **kwargs):
        layer_szs = [body_in // expansion] + layer_szs + [body_out]
        num_layers = len(layer_szs) - 1
        layers = [(f"layer_{i}", bodylayer(block, blocks[i],
                                           layer_szs[i], layer_szs[i + 1], expansion,
                                           stride=1 if i == 0 else 2,
                                           sa=sa if i == 0 else False,
                                           **kwargs))
                  for i in range(num_layers)]
        super().__init__(OrderedDict(layers))

In [ ]:
body = Body(BasicBlock)
body

Body(
  (layer_0): BasicLayer(
    from 64 to 64, 2 blocks, expansion 1.
    (block_0): BasicBlock(
      (conv): Sequential(
        (conv_0): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act_fn): ReLU(inplace=True)
        )
        (conv_1): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (merge): Noop()
      (act_conn): ReLU(inplace=True)
    )
    (block_1): BasicBlock(
      (conv): Sequential(
        (conv_0): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act_fn): ReLU(i

In [ ]:
#hide
body = Body(BasicBlock, bn_1st=False)
body

Body(
  (layer_0): BasicLayer(
    from 64 to 64, 2 blocks, expansion 1.
    (block_0): BasicBlock(
      (conv): Sequential(
        (conv_0): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (act_fn): ReLU(inplace=True)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (conv_1): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (merge): Noop()
      (act_conn): ReLU(inplace=True)
    )
    (block_1): BasicBlock(
      (conv): Sequential(
        (conv_0): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (act_fn): ReLU(inplace=True)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running

In [ ]:
bs = 8
xb = torch.randn(bs, 64, 32, 32)
y = body(xb)
print(y.shape)
assert y.shape==torch.Size([bs, 512, 4, 4])

torch.Size([8, 512, 4, 4])


In [ ]:
#hide
body = Body(BasicBlock, sa=True)
body.layer_0

BasicLayer(
  from 64 to 64, 2 blocks, expansion 1.
  (block_0): BasicBlock(
    (conv): Sequential(
      (conv_0): ConvLayer(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act_fn): ReLU(inplace=True)
      )
      (conv_1): ConvLayer(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (merge): Noop()
    (act_conn): ReLU(inplace=True)
  )
  (block_1): BasicBlock(
    (conv): Sequential(
      (conv_0): ConvLayer(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act_fn): ReLU(inplace=True)
      )
      (conv_1): ConvLayer(
        (conv):

In [ ]:
#hide
bs = 8
xb = torch.randn(bs, 64, 32, 32)
y = body(xb)
print(y.shape)
assert y.shape==torch.Size([bs, 512, 4, 4])

torch.Size([8, 512, 4, 4])


# Head

In [ ]:
#export
class Head(nn.Sequential):
    '''base head'''
    def __init__(self, ni, nf, **kwargs):
        super().__init__(OrderedDict(
            [('pool', nn.AdaptiveAvgPool2d((1, 1))),
             ('flat', Flatten()),
             ('fc', nn.Linear(ni, nf)),
             ]))

In [ ]:
head = Head(512, 10)

In [ ]:
bs = 8
xb = torch.randn(bs, 512, 4, 4)
y = head(xb)
y.shape

torch.Size([8, 10])

# class Net

In [ ]:
#export
def init_model(model, nonlinearity='leaky_relu'):
    '''Init model'''
    for m in model.modules():
        if isinstance(m, (nn.Conv2d, nn.Linear)):
            nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity=nonlinearity)

In [ ]:
#export
class Net(nn.Sequential):
    '''Constructor for model'''
    def __init__(self, stem=Stem,
                 body=Body, block=BasicBlock, sa=False,
                 layer_szs=[64, 128, 256, ], blocks=[2, 2, 2, 2],
                 head=Head,
                 c_in=3, num_classes=1000,
                 body_in=64, body_out=512, expansion=1,
                 init_fn=init_model, **kwargs):
        self.init_model = init_fn
        super().__init__(OrderedDict(
            [('stem', stem(c_in=c_in, stem_out=body_in, **kwargs)),
             ('body', body(block, body_in, body_out,
                           layer_szs=layer_szs, blocks=blocks, expansion=expansion,
                           sa=sa, **kwargs)),
             ('head', head(body_out * expansion, num_classes, **kwargs))
             ]))
        self.init_model(self)

In [ ]:
model = Net()

In [ ]:
model

Net(
  (stem): Stem(
    sizes: [3, 64]
    (conv_0): ConvLayer(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (body): Body(
    (layer_0): BasicLayer(
      from 64 to 64, 2 blocks, expansion 1.
      (block_0): BasicBlock(
        (conv): Sequential(
          (conv_0): ConvLayer(
            (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (act_fn): ReLU(inplace=True)
          )
          (conv_1): ConvLayer(
            (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=T

In [ ]:
bs_test = 16
xb = torch.randn(bs_test, 3, 128, 128)
y = model(xb)
y.shape

torch.Size([16, 1000])

In [ ]:
#hide
model = Net(bn_1st=False)
model.body.layer_0

BasicLayer(
  from 64 to 64, 2 blocks, expansion 1.
  (block_0): BasicBlock(
    (conv): Sequential(
      (conv_0): ConvLayer(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (act_fn): ReLU(inplace=True)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv_1): ConvLayer(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (merge): Noop()
    (act_conn): ReLU(inplace=True)
  )
  (block_1): BasicBlock(
    (conv): Sequential(
      (conv_0): ConvLayer(
        (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (act_fn): ReLU(inplace=True)
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (conv_1): ConvLayer(
        (conv):

In [ ]:
#hide
bs_test = 8
xb = torch.randn(bs_test, 3, 128, 128)
y = model(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 1000]), f"size expected {bs_test}, 1000"

torch.Size([8, 1000])


In [ ]:
#hide
model = Net(sa=True)
model

Net(
  (stem): Stem(
    sizes: [3, 64]
    (conv_0): ConvLayer(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (body): Body(
    (layer_0): BasicLayer(
      from 64 to 64, 2 blocks, expansion 1.
      (block_0): BasicBlock(
        (conv): Sequential(
          (conv_0): ConvLayer(
            (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (act_fn): ReLU(inplace=True)
          )
          (conv_1): ConvLayer(
            (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=T

In [ ]:
#hide
bs_test = 8
xb = torch.randn(bs_test, 3, 128, 128)
y = model(xb)
print(y.shape)
assert y.shape == torch.Size([bs_test, 1000]), f"size expected {bs_test}, 1000"

torch.Size([8, 1000])


# model constructor
by ayasyrev

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_Net.ipynb.
Converted 01_activations.ipynb.
Converted 01_layers.ipynb.
Converted 03_MXResNet.ipynb.
Converted 04_YaResNet.ipynb.
Converted 05_Twist.ipynb.
Converted 10_base_constructor.ipynb.
Converted 11_xresnet.ipynb.
Converted index.ipynb.
